In [2]:
import pandas as pd
import numpy as np
import requests
import time
import json

In [3]:
df=pd.read_csv('processed data.csv')
df.head()

,Bus,Route number,Institute,Driver_Lat,Driver_Lon,PP_Lat,PP_Lon,Dead km,Dead_KM_Haversine
0,KA53AC0634,1,Chanakya,12.958330,77.525384,12.958330,77.525384,0.0,0.000000
1,KA53AC0540,2,Chanakya,13.249466,77.708713,13.249466,77.708713,0.0,0.000000
2,KA439840,1,Gitam,13.017359,77.657409,13.010978,77.695193,6.0,4.154431
3,KA43A0245,2,Gitam,13.033098,77.553878,12.967800,77.536097,6.0,7.512091
4,KA439841,3,Gitam,13.099021,77.574234,13.034584,77.557486,8.0,7.391135


In [4]:
base_url = "http://localhost:5000/route/v1/driving"
forward_dists, reverse_dists = [], []

In [21]:
# for _, row in df.iterrows():
#     src = f"{row['Driver_Lon']},{row['Driver_Lat']};{row['PP_Lon']},{row['PP_Lat']}"
    # rev = f"{row['PP_Lon']},{row['PP_Lat']};{row['Driver_Lon']},{row['Driver_Lat']}"
    # try:
    #     f_resp = requests.get(f"{base_url}/{src}?overview=false")
    #     forward = f_resp.json()['routes'][0]['distance'] / 1000 if f_resp.ok else None
    # except:
    #     forward = None

    # try:
    #     r_resp = requests.get(f"{base_url}/{rev}?overview=false")
    #     reverse = r_resp.json()['routes'][0]['distance'] / 1000 if r_resp.ok else None
    # except:
    #     reverse = None

    # forward_dists.append(forward)
    # reverse_dists.append(reverse)
    # time.sleep(0.2)

    # df['distance_1_to_2'] = forward_dists
    # df['distance_2_to_1'] = reverse_dists

def osrm_dist(lon1, lat1, lon2, lat2):
    src=f'{lon1},{lat1};{lon2},{lat2}'
    res=requests.get(f"http://localhost:5000/route/v1/driving/{src}?overview=false")
    # forward = res.json()['routes'][0]['distance'] / 1000
    # print(res.json())
    forward=res.json()['routes'][0]['distance'] / 1000
    return forward

# print(osrm_dist(77.65740947,13.0173592,77.63069568,13.03201432))
df['OS_Distance'] = df.apply(
    lambda row: osrm_dist(row['Driver_Lon'], row['Driver_Lat'], row['PP_Lon'], row['PP_Lat']),
    axis=1
)



{'code': 'Ok', 'routes': [{'legs': [{'steps': [], 'weight': 0, 'summary': '', 'duration': 0, 'distance': 0}], 'weight_name': 'routability', 'weight': 0, 'duration': 0, 'distance': 0}], 'waypoints': [{'hint': '87yjgFm9o4ANAAAAHQAAAAAAAAAAAAAAdCkVQd9VoEEAAAAAAAAAAA0AAAAdAAAAAAAAAAAAAADXAgAAXfGeBIi6xQCI8Z4EerrFAAAALwEAAAAA', 'location': [77.525341, 12.958344], 'name': '', 'distance': 4.915687327}, {'hint': '87yjgFm9o4ANAAAAHQAAAAAAAAAAAAAAdCkVQd9VoEEAAAAAAAAAAA0AAAAdAAAAAAAAAAAAAADXAgAAXfGeBIi6xQCI8Z4EerrFAAAALwEAAAAA', 'location': [77.525341, 12.958344], 'name': '', 'distance': 4.915687327}]}
{'code': 'Ok', 'routes': [{'legs': [{'steps': [], 'weight': 0, 'summary': '', 'duration': 0, 'distance': 0}], 'weight_name': 'routability', 'weight': 0, 'duration': 0, 'distance': 0}], 'waypoints': [{'hint': 'eh4PgK_8m4AZAAAANgAAAFkAAAAAAAAAPHyNQcILFEIa0ndCAAAAABkAAAA2AAAAWQAAAAAAAADXAgAAr72hBN0rygCpvaEEuivKAAIAPwYAAAAA', 'location': [77.708719, 13.249501], 'name': '', 'distance': 3.926364487}, {'hi

In [22]:
df.head()

,Bus,Route number,Institute,Driver_Lat,Driver_Lon,PP_Lat,PP_Lon,Dead km,Dead_KM_Haversine,OSRM_Distance,OS_Distance
0,KA53AC0634,1,Chanakya,12.958330,77.525384,12.958330,77.525384,0.0,0.000000,785.0128,0.0000
1,KA53AC0540,2,Chanakya,13.249466,77.708713,13.249466,77.708713,0.0,0.000000,848.6495,0.0000
2,KA439840,1,Gitam,13.017359,77.657409,13.010978,77.695193,6.0,4.154431,826.9587,5.9652
3,KA43A0245,2,Gitam,13.033098,77.553878,12.967800,77.536097,6.0,7.512091,825.2217,9.8034
4,KA439841,3,Gitam,13.099021,77.574234,13.034584,77.557486,8.0,7.391135,833.5375,10.0293


In [23]:
df.to_csv('processed_data with osrm distance.csv', index=False)

In [24]:
print(osrm_dist(77.60156932,12.98517583,77.60156932,12.98517583))

{'code': 'Ok', 'routes': [{'legs': [{'steps': [], 'weight': 0, 'summary': '', 'duration': 0, 'distance': 0}], 'weight_name': 'routability', 'weight': 0, 'duration': 0, 'distance': 0}], 'waypoints': [{'hint': 'MoamgHKGpoAgAAAAMgAAAAAAAAADAAAA63K0QUrDCUIAAAAAA-4CQCAAAAAyAAAAAAAAAAMAAADXAgAAPhugBNkjxgAhG6AEWCPGAAAArxEAAAAA', 'location': [77.601598, 12.985305], 'name': '', 'distance': 14.61402803}, {'hint': 'MoamgHKGpoAgAAAAMgAAAAAAAAADAAAA63K0QUrDCUIAAAAAA-4CQCAAAAAyAAAAAAAAAAMAAADXAgAAPhugBNkjxgAhG6AEWCPGAAAArxEAAAAA', 'location': [77.601598, 12.985305], 'name': '', 'distance': 14.61402803}]}
0.0
